# Don't use this -- currently combining `ipyparallel` and `numba` deadlocks.

# Using `numba` with `ipyparallel`

First make sure `ipyparallel` is installed 

```console
conda install ipyparallel
```

Then spin up a local 'cluster' with 

```console
ipcluster start -n 4
```

(assuming you have 4 cores)

In [ ]:
from ipyparallel import Client

Check that you have 4 clients (or however many workers you have started with `ipcluster`)

In [ ]:
rc = Client()
rc.ids

Create a 'direct view' of all workers

In [ ]:
dv = rc[:]

## Mandelbrot serial pure python

Define pure Python mandelbrot function

In [ ]:
def mandel(x, y):
    max_iters = 20
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if z.real * z.real + z.imag * z.imag >= 4:
            return i
    return 255

## Mandelbrot `ipyparallel` pure python

Add the `dv.parallel` decorator to enable it in parallel.  (We're also enabling blocking here, for simplicity)

In [ ]:
@dv.parallel(block=True)
def mandel_par(x, y):
    max_iters = 20
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if z.real * z.real + z.imag * z.imag >= 4:
            return i
    return 255

## Mandelbrot `ipyparallel` & `numba`

To use `numba` with the parallel version, simply stack the decorators

In [ ]:
from numba import jit

In [ ]:
@dv.parallel(block=True)
@jit(nopython=True)
def mandel_par_numba(x, y):
    max_iters = 20
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if z.real * z.real + z.imag * z.imag >= 4:
            return i
    return 255

In [ ]:
import numpy

In [ ]:
x = numpy.arange(-2, 1, 0.005)
y = numpy.arange(-1, 1, 0.005)
X, Y = numpy.meshgrid(x, y)

## 1 core

In [ ]:
%%time
im = numpy.reshape(list(map(mandel, X.ravel(), Y.ravel())), (len(y), len(x)))

## 4 cores with `ipyparallel`

In [ ]:
%%time
im_par = numpy.reshape(mandel_par.map(X.ravel(), Y.ravel()), (len(y), len(x)))

## 4 cores with `ipyparallel` and `numba`

In [ ]:
%%time
im_par_numba = numpy.reshape(mandel_par_numba.map(X.ravel(), Y.ravel()), (len(y), len(x)))

In [ ]:
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
fig, axes = pyplot.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(im, cmap=cm.viridis)
axes[1].imshow(im_par, cmap=cm.viridis)
axes[2].imshow(im_par_numba, cmap=cm.viridis)

Original `ipyparallel` example taken from Duke's [Computational Statistics in Python](http://people.duke.edu/~ccc14/sta-663-2016/19C_IPyParallel.html#Example:-Use-the-@parallel-decorator-to-speed-up-Mandelbrot-calculations) course